In [4]:
import math
from tqdm import tqdm
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GroupKFold

# Visuals and CV2





import cv2

# albumentations for augs
import albumentations
from albumentations.pytorch.transforms import ToTensorV2

from sklearn.model_selection import KFold, train_test_split

#torch
import torch
import torch.nn as nn
from torch.nn import Parameter
from torch.nn import functional as F
from torch.utils.data import Dataset,DataLoader
from torch.optim import Adam
from torch.optim.lr_scheduler import _LRScheduler
from torch.optim import Adam, lr_scheduler

import transformers
from transformers import AdamW
from transformers import get_linear_schedule_with_warmup,get_cosine_schedule_with_warmup
from transformers import get_cosine_with_hard_restarts_schedule_with_warmup
import transformers
from transformers import (BertTokenizer, BertModel,
                          DistilBertTokenizer, DistilBertModel, XLMRobertaTokenizer, XLMRobertaModel)

# Configuration

In [5]:
NUM_WORKERS = 4
TRAIN_BATCH_SIZE = 16
EPOCHS = 50
SEED = 2020
LR = 5e-5

device = torch.device('cuda')

################################################# MODEL ####################################################################

TOKENIZER = XLMRobertaTokenizer.from_pretrained("xlm-roberta-base")
transformer_model = XLMRobertaModel.from_pretrained("xlm-roberta-base")

################################################ Metric Loss and its params #######################################################
loss_module = 'arcface'
s = 30.0
m = 0.5 
ls_eps = 0.0
easy_margin = False

############################################################################################################################
model_params = {
    'n_classes':46,
    'model_name':transformer_model,
    'pooling':'mean',
    'use_fc':False,
    'fc_dim':512,
    'dropout':0.0,
    'loss_module':loss_module,
    's':30.0,
    'margin':0.50,
    'ls_eps':0.0,
    'theta_zero':0.785
}

Downloading:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/512 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

# Utils

In [6]:
class AvgMeter:
    def __init__(self, name="Metric"):
        self.name = name
        self.reset()
    
    def reset(self):
        self.avg, self.sum, self.count = [0]*3
    
    def update(self, val, count=1):
        self.count += count
        self.sum += val * count
        self.avg = self.sum / self.count
    
    def __repr__(self):
        text = f"{self.name}: {self.avg:.4f}"
        return text

    
def get_accuracy(preds, targets):
    """
    preds shape: (batch_size, num_labels)
    targets shape: (batch_size)
    """
    preds = preds.argmax(dim=1)
    acc = (preds == targets).float().mean()
    return acc

# Dataset preparation

In [7]:
train = pd.read_csv("../input/russe-data/russe-wsi-kit-master/data/main/bts-rnc/train.csv", sep='\t')
train.head()

,context_id,word,gold_sense_id,predict_sense_id,positions,context
0,1,балка,1,NaN,90-94,"маленькой комнаты. Он был очень высок, наклони..."
1,2,балка,1,NaN,69-73,Пантюхин в Склифе сейчас. Он выползти на улицу...
2,3,балка,1,NaN,115-121,равнозначно обеспечивает и меланхоличную езду....
3,4,балка,1,NaN,85-90,"верхняя часть закрыта, замкнута, многократно о..."
4,5,балка,1,NaN,66-70,"по телевизору: наши гол забили, я вскочил от р..."


In [8]:
train

,context_id,word,gold_sense_id,predict_sense_id,positions,context
0,1,балка,1,NaN,90-94,"маленькой комнаты. Он был очень высок, наклони..."
1,2,балка,1,NaN,69-73,Пантюхин в Склифе сейчас. Он выползти на улицу...
2,3,балка,1,NaN,115-121,равнозначно обеспечивает и меланхоличную езду....
3,4,балка,1,NaN,85-90,"верхняя часть закрыта, замкнута, многократно о..."
4,5,балка,1,NaN,66-70,"по телевизору: наши гол забили, я вскочил от р..."
...,...,...,...,...,...,...
3486,3487,штамп,4,NaN,81-86,"дурак, но партию свою отрабатывал точно, по ус..."
3487,3488,штамп,4,NaN,85-90,"дело... получается, мыслить и выражать свое мн..."
3488,3489,штамп,4,NaN,71-77,", что актер должен иметь пять штампов-клише ""п..."
3489,3490,штамп,4,NaN,107-112,сегодняшний день в системе негосударственного ...


In [9]:
d = {}
r = 0
tmp = []
for i, row in train.iterrows():
    if (row.word, row.gold_sense_id) not in d.keys():
        d[(row.word, row.gold_sense_id)] = r
        tmp.append(r)
        r = r + 1
    else:
        tmp.append(d[(row.word, row.gold_sense_id)])

In [10]:
train['labels'] = tmp    # Get class of a context

In [11]:
train

,context_id,word,gold_sense_id,predict_sense_id,positions,context,labels
0,1,балка,1,NaN,90-94,"маленькой комнаты. Он был очень высок, наклони...",0
1,2,балка,1,NaN,69-73,Пантюхин в Склифе сейчас. Он выползти на улицу...,0
2,3,балка,1,NaN,115-121,равнозначно обеспечивает и меланхоличную езду....,0
3,4,балка,1,NaN,85-90,"верхняя часть закрыта, замкнута, многократно о...",0
4,5,балка,1,NaN,66-70,"по телевизору: наши гол забили, я вскочил от р...",0
...,...,...,...,...,...,...,...
3486,3487,штамп,4,NaN,81-86,"дурак, но партию свою отрабатывал точно, по ус...",95
3487,3488,штамп,4,NaN,85-90,"дело... получается, мыслить и выражать свое мн...",95
3488,3489,штамп,4,NaN,71-77,", что актер должен иметь пять штампов-клише ""п...",95
3489,3490,штамп,4,NaN,107-112,сегодняшний день в системе негосударственного ...,95


In [12]:
d = {}
r = 0
tmp = []
for i, row in train.iterrows():
    if row.labels not in d.keys():
        d[row.labels] = 1
    else:
        d[row.labels] = d[row.labels] + 1

In [13]:
tmp = []
for i, row in train.iterrows():
    tmp.append(d[row.labels])                     #get occurence of each class

In [14]:
train['occurences'] = tmp

**Split Data**

In [15]:
group_kfold = GroupKFold(n_splits=3)
group_kfold.get_n_splits(train, train.word, train.word)

3

In [16]:
for train_index, test_index in group_kfold.split(train, train.word, train.word):
    X_train, X_test = train.iloc[train_index], train.iloc[test_index]
    y_train, y_test = train.labels[train_index], train.labels[test_index]

In [17]:
len(X_train.word.unique())

20

In [18]:
len(train.word.unique())

30

In [19]:
len(X_test.word.unique())

10

In [20]:
train = X_train[X_train['occurences'] >= 5]

In [21]:
train

,context_id,word,gold_sense_id,predict_sense_id,positions,context,labels,occurences
0,1,балка,1,NaN,90-94,"маленькой комнаты. Он был очень высок, наклони...",0,81
1,2,балка,1,NaN,69-73,Пантюхин в Склифе сейчас. Он выползти на улицу...,0,81
2,3,балка,1,NaN,115-121,равнозначно обеспечивает и меланхоличную езду....,0,81
3,4,балка,1,NaN,85-90,"верхняя часть закрыта, замкнута, многократно о...",0,81
4,5,балка,1,NaN,66-70,"по телевизору: наши гол забили, я вскочил от р...",0,81
...,...,...,...,...,...,...,...,...
3486,3487,штамп,4,NaN,81-86,"дурак, но партию свою отрабатывал точно, по ус...",95,47
3487,3488,штамп,4,NaN,85-90,"дело... получается, мыслить и выражать свое мн...",95,47
3488,3489,штамп,4,NaN,71-77,", что актер должен иметь пять штампов-клише ""п...",95,47
3489,3490,штамп,4,NaN,107-112,сегодняшний день в системе негосударственного ...,95,47


In [22]:
train.word.unique()

array(['балка', 'вид', 'винт', 'горн', 'клетка', 'купюра', 'курица',
       'лира', 'мина', 'мишень', 'оклад', 'полис', 'пост', 'проказа',
       'пропасть', 'проспект', 'рысь', 'среда', 'хвост', 'штамп'],
      dtype=object)

In [23]:
test = pd.read_csv("../input/russe-data/russe-wsi-kit-master/data/main/bts-rnc/test.csv", sep='\t')
test.head()

,context_id,word,gold_sense_id,predict_sense_id,positions,context
0,3492,акция,NaN,NaN,104-108,"точными сведениями осенью. Напомним, что снача..."
1,3493,акция,NaN,NaN,79-83,"пакет акций. «Все зависит от того, насколько р..."
2,3494,акция,NaN,NaN,70-74,"в ЕГРЮЛ только 5 октября 2012 года, а госрегис..."
3,3495,акция,NaN,NaN,67-71,"8%, а бумаги Дальсвязи подорожали на 4,4%. Вме..."
4,3496,акция,NaN,NaN,90-94,", пожилые - у нас много категорий граждан, кот..."


In [24]:
train.columns

Index(['context_id', 'word', 'gold_sense_id', 'predict_sense_id', 'positions',
       'context', 'labels', 'occurences'],
      dtype='object')

In [25]:
def get_ixs(df):
    get = []
    for index, row in df.iterrows():
        cont = row['context']
        pos = int(row['positions'].split('-')[0])
        #print(pos)
        cont = cont.split(' ')
        cnt = 0
        res = 0
        while cnt < pos:
            cnt += len(cont[res]) + 1
            res = res + 1
        get.append(res)
    return get

In [26]:
train['indice'] = get_ixs(train)
X_test['indice'] = get_ixs(X_test)
test['indice'] = get_ixs(test)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


**DataLoader**

In [27]:
class WiCFeature2:
    def __init__(self, input_ids, input_mask, token_type_ids, label, positions, example):
        self.input_ids = input_ids
        self.input_mask = input_mask
        self.token_type_ids = token_type_ids
        self.label = label
        self.positions = positions
        self.example = example

In [28]:
train

,context_id,word,gold_sense_id,predict_sense_id,positions,context,labels,occurences,indice
0,1,балка,1,NaN,90-94,"маленькой комнаты. Он был очень высок, наклони...",0,81,12
1,2,балка,1,NaN,69-73,Пантюхин в Склифе сейчас. Он выползти на улицу...,0,81,12
2,3,балка,1,NaN,115-121,равнозначно обеспечивает и меланхоличную езду....,0,81,12
3,4,балка,1,NaN,85-90,"верхняя часть закрыта, замкнута, многократно о...",0,81,12
4,5,балка,1,NaN,66-70,"по телевизору: наши гол забили, я вскочил от р...",0,81,12
...,...,...,...,...,...,...,...,...,...
3486,3487,штамп,4,NaN,81-86,"дурак, но партию свою отрабатывал точно, по ус...",95,47,12
3487,3488,штамп,4,NaN,85-90,"дело... получается, мыслить и выражать свое мн...",95,47,13
3488,3489,штамп,4,NaN,71-77,", что актер должен иметь пять штампов-клише ""п...",95,47,12
3489,3490,штамп,4,NaN,107-112,сегодняшний день в системе негосударственного ...,95,47,12


In [29]:
train['positions'].str.split('-')[0][0]

'90'

In [30]:
lst1 = []
lst2 = []
for i, ex in train.iterrows():
    pos1 = ex.positions.split('-')[0]
    pos2 = ex.positions.split('-')[1]
    lst1.append(int(pos1))
    lst2.append(int(pos2))

In [31]:
train['start_1'] = lst1
train['end_1'] = lst2

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [32]:
train

,context_id,word,gold_sense_id,predict_sense_id,positions,context,labels,occurences,indice,start_1,end_1
0,1,балка,1,NaN,90-94,"маленькой комнаты. Он был очень высок, наклони...",0,81,12,90,94
1,2,балка,1,NaN,69-73,Пантюхин в Склифе сейчас. Он выползти на улицу...,0,81,12,69,73
2,3,балка,1,NaN,115-121,равнозначно обеспечивает и меланхоличную езду....,0,81,12,115,121
3,4,балка,1,NaN,85-90,"верхняя часть закрыта, замкнута, многократно о...",0,81,12,85,90
4,5,балка,1,NaN,66-70,"по телевизору: наши гол забили, я вскочил от р...",0,81,12,66,70
...,...,...,...,...,...,...,...,...,...,...,...
3486,3487,штамп,4,NaN,81-86,"дурак, но партию свою отрабатывал точно, по ус...",95,47,12,81,86
3487,3488,штамп,4,NaN,85-90,"дело... получается, мыслить и выражать свое мн...",95,47,13,85,90
3488,3489,штамп,4,NaN,71-77,", что актер должен иметь пять штампов-клише ""п...",95,47,12,71,77
3489,3490,штамп,4,NaN,107-112,сегодняшний день в системе негосударственного ...,95,47,12,107,112


In [33]:
def convert_dataset_to_features(
            df
    ):
        features = []
        max_seq_len = 128

        num_too_long_exs = 0
        for (ex_index, ex) in df.iterrows():
            label = ex.labels
            st1, end1  = int(ex.start_1), int(ex.end_1) + 1
            sent1 = ex.context
            tokens = [TOKENIZER.cls_token]
            left1, target1, right1 = sent1[:st1], sent1[st1:end1], sent1[end1:]
            #print(target1)
            if left1:
                tokens += TOKENIZER.tokenize(left1)
            st = len(tokens)
            target_subtokens = TOKENIZER.tokenize(target1)
            tokens += target_subtokens
            end = len(tokens)

            if right1:
                tokens += TOKENIZER.tokenize(right1)

            input_ids = TOKENIZER.convert_tokens_to_ids(tokens)
            if len(input_ids) > max_seq_len:
                input_ids = input_ids[:max_seq_len]
                num_too_long_exs += 1
                if max(positions) > max_seq_len - 1:
                    continue

            input_mask = [1] * len(input_ids)
            token_type_ids = [0] * max_seq_len
            padding = [TOKENIZER.convert_tokens_to_ids([TOKENIZER.pad_token])[0]] * (max_seq_len - len(input_ids))
            input_ids += padding
            input_mask += [0] * len(padding)

            features.append(
                WiCFeature2(
                    input_ids=input_ids,
                    input_mask=input_mask,
                    token_type_ids=token_type_ids,
                    label=label,
                    positions=(st, end),
                    example=ex
                )
            )
        print("Not fitted examples percentage: %s" % str(num_too_long_exs / len(features) * 100.0))
        return features

In [34]:
feats = convert_dataset_to_features(train)

Not fitted examples percentage: 0.0


In [35]:
class WSIDataset(Dataset):
    def __init__(self, df, feats):
        self.csv = df
        self.feats = feats
        #self.positions = list(csv['indice'])
        
    def __len__(self):
        return len(self.csv)

    def __getitem__(self, index):
        row = self.csv.iloc[index]
        feat = self.feats[index]
        
        
        text = row.context
        
        text = TOKENIZER(text, padding='max_length', truncation=True, max_length=128, return_tensors="pt")
        #item['positions'] = self.positions[index]
        
        input_ids = text['input_ids'][0]
        #print(input_ids)
        attention_mask = text['attention_mask'][0]  
        
        return input_ids, attention_mask, torch.tensor(feat.label), feat.positions

# Metric Learning Losses

https://github.com/lyakaap/Landmark2019-1st-and-3rd-Place-Solution/blob/master/src/modeling/metric_learning.py

In [36]:
# ArcFace Loss
class ArcMarginProduct(nn.Module):
    def __init__(self, in_features, out_features, s=30.0, m=0.50, easy_margin=False, ls_eps=0.0):
        super(ArcMarginProduct, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.s = s
        self.m = m
        self.ls_eps = ls_eps  # label smoothing
        self.weight = Parameter(torch.FloatTensor(out_features, in_features))
        nn.init.xavier_uniform_(self.weight)

        self.easy_margin = easy_margin
        self.cos_m = math.cos(m)
        self.sin_m = math.sin(m)
        self.th = math.cos(math.pi - m)
        self.mm = math.sin(math.pi - m) * m

    def forward(self, input, label):
        # --------------------------- cos(theta) & phi(theta) ---------------------------
        cosine = F.linear(F.normalize(input), F.normalize(self.weight))
        sine = torch.sqrt(1.0 - torch.pow(cosine, 2))
        phi = cosine * self.cos_m - sine * self.sin_m
        if self.easy_margin:
            phi = torch.where(cosine > 0, phi, cosine)
        else:
            phi = torch.where(cosine > self.th, phi, cosine - self.mm)
        # --------------------------- convert label to one-hot ---------------------------
        # one_hot = torch.zeros(cosine.size(), requires_grad=True, device='cuda')
        one_hot = torch.zeros(cosine.size(), device='cuda')
        one_hot.scatter_(1, label.view(-1, 1).long(), 1)
        if self.ls_eps > 0:
            one_hot = (1 - self.ls_eps) * one_hot + self.ls_eps / self.out_features
        # -------------torch.where(out_i = {x_i if condition_i else y_i) -------------
        output = (one_hot * phi) + ((1.0 - one_hot) * cosine)
        output *= self.s

        return output

# Model

In [37]:

class WSINet(nn.Module):

    def __init__(self,
                 n_classes,
                 model_name='xlm-roberta-base',
                 pooling='mean_pooling',
                 use_fc=False,
                 fc_dim=512,
                 dropout=0.0,
                 loss_module='softmax',
                 s=30.0,
                 margin=0.50,
                 ls_eps=0.0,
                 theta_zero=0.785):
        """
        :param n_classes:
        :param model_name: name of model from pretrainedmodels
            e.g. resnet50, resnext101_32x4d, pnasnet5large
        :param pooling: One of ('SPoC', 'MAC', 'RMAC', 'GeM', 'Rpool', 'Flatten', 'CompactBilinearPooling')
        :param loss_module: One of ('arcface', 'cosface', 'softmax')
        """
        super(WSINet, self).__init__()

        self.transformer = XLMRobertaModel.from_pretrained("xlm-roberta-base")
        final_in_features = self.transformer.config.hidden_size
        
        self.pooling = pooling
        self.use_fc = use_fc
    
        if use_fc:
            self.dropout = nn.Dropout(p=dropout)
            self.fc = nn.Linear(final_in_features, fc_dim)
            self.bn = nn.BatchNorm1d(fc_dim)
            self.relu = nn.ReLU()
            self._init_params()
            final_in_features = fc_dim

        self.loss_module = loss_module
        if loss_module == 'arcface':
            self.final = ArcMarginProduct(final_in_features, n_classes,
                                          s=s, m=margin, easy_margin=False, ls_eps=ls_eps)
        else:
            self.final = nn.Linear(final_in_features, n_classes)

    def _init_params(self):
        nn.init.xavier_normal_(self.fc.weight)
        nn.init.constant_(self.fc.bias, 0)
        nn.init.constant_(self.bn.weight, 1)
        nn.init.constant_(self.bn.bias, 0)

    def forward(self, input_ids,attention_mask, label, position):
        x = self.transformer(input_ids=input_ids,attention_mask=attention_mask)
        #x = x[0]
        #print(x.size())
        feature = self.extract_features(x, position)
        
        if self.loss_module == 'arcface':
            logits = self.final(feature, label)
        else:
            logits = self.final(feature)
        return logits
    
    def extract_features(self, hidden_states, positions):
        pool_type = model_params['pooling']
        #merge_type = self.local_config['target_embeddings']
        #print(hidden_states)
        hidden_states = hidden_states.last_hidden_state 
        #print(type(hidden_states))
        bs, seq, hs = hidden_states.size()
        features = []
        #print(f"Positions:{positions}")
        for ex_id in range(bs):
            #print(type(positions[ex_id, 0].item()))
            #print(positions[ex_id, 0].item())
            #print(type(ex_id))
            #print(positions[ex_id])
            start1, end1 = positions[0][ex_id], positions[1][ex_id]
            if pool_type == 'mean':
                emb1 = hidden_states[ex_id, start1:end1].mean(dim=0) # hidden
                #emb2 = hidden_states[ex_id, start2:end2].mean(dim=0) # hidden
            elif pool_type == 'max':
                emb1, _ = hidden_states[ex_id, start1:end1].max(dim=0) # hidden
                #emb2, _ = hidden_states[ex_id, start2:end2].max(dim=0) # hidden
            elif pool_type == 'first':
                emb1 = hidden_states[ex_id, start1]
                #emb2 = hidden_states[ex_id, start2]
            elif pool_type == 'mf':
                embs1 = hidden_states[ex_id, start1:end1] # hidden
                #embs2 = hidden_states[ex_id, start2:end2] # hidden
                #emb1, emb2 = (torch.cat([embs.mean(dim=0), embs[0]], dim=-1) for embs in (embs1, embs2))
            elif pool_type.startswith('mm'):
                embs1 = hidden_states[ex_id, start1:end1] # hidden
                #embs2 = hidden_states[ex_id, start2:end2] # hidden
                #last = '' if len(pool_type)<3 else pool_type[2]
                #emb1, emb2 = (torch.cat([embs.mean(dim=0), embs.max(dim=0).values] + ([] if last=='' else [embs[0]] if last=='f' else [embs.min(dim=0).values]), dim=-1) for embs in (embs1, embs2))
            else:
                raise ValueError(f'wrong pool_type: {pool_type}')
            """
            if merge_type == 'featwise_mul':
                merged_feature = emb1 * emb2 # hidden
            elif merge_type == 'diff':
                merged_feature = emb1 - emb2
            elif merge_type == 'concat':
                merged_feature = torch.cat((emb1, emb2)) # 2 * hidden
            elif merge_type == 'mulnorm':
                merged_feature = (emb1 / emb1.norm(dim=-1, keepdim=True)) * (emb2 / emb2.norm(dim=-1, keepdim=True))
            elif merge_type == 'comb_cm':
                merged_feature = torch.cat((emb1, emb2, emb1 * emb2))
            elif merge_type == 'comb_cmn':
                emb1n = emb1 / emb1.norm(dim=-1, keepdim=True)
                emb2n = emb2 / emb2.norm(dim=-1, keepdim=True)
                merged_feature = torch.cat((emb1, emb2, emb1n*emb2n))
            elif merge_type == 'comb_cd':
                merged_feature = torch.cat((emb1, emb2, emb1 - emb2))
            elif merge_type == 'comb_cnmn':
                emb1n = emb1 / emb1.norm(dim=-1, keepdim=True)
                emb2n = emb2 / emb2.norm(dim=-1, keepdim=True)
                merged_feature = torch.cat((emb1n, emb2n, emb1n* emb2n))
            elif merge_type == 'comb_dmn':
                emb1n = emb1 / emb1.norm(dim=-1, keepdim=True)
                emb2n = emb2 / emb2.norm(dim=-1, keepdim=True)
                merged_feature = torch.cat((emb1 - emb2, emb1n* emb2n))
            elif merge_type == 'comb_dnmn':
                emb1n = emb1 / emb1.norm(dim=-1, keepdim=True)
                emb2n = emb2 / emb2.norm(dim=-1, keepdim=True)
                merged_feature = torch.cat((emb1n - emb2n, emb1n* emb2n))
            elif merge_type.startswith( 'dist_'):
                if 'n' in merge_type:
                    emb1n = emb1 / emb1.norm(dim=-1, keepdim=True)
                    emb2n = emb2 / emb2.norm(dim=-1, keepdim=True)
                dists = []
                if 'l1' in merge_type:        
                    dists.append(torch.norm(emb1n-emb2n if 'l1n' in merge_type else emb1-emb2, dim=-1, p=1, keepdim=True))
                if 'l2' in merge_type:        
                    dists.append(torch.norm(emb1n-emb2n if 'l2n' in merge_type else emb1-emb2, dim=-1, p=2, keepdim=True))
                if 'dot' in merge_type:        
                    dists.append((emb1n*emb2n if 'dotn' in merge_type else emb1*emb2 ).sum(dim=-1, keepdim=True))
                merged_feature = torch.cat(dists)
            """

            features.append(emb1.unsqueeze(0))
        output = torch.cat(features, dim=0) # bs x hidden
        return output


class WSINet(nn.Module):

    def __init__(self,
                 n_classes,
                 model_name='bert-base-uncased',
                 pooling='mean_pooling',
                 use_fc=False,
                 fc_dim=512,
                 dropout=0.0,
                 loss_module='softmax',
                 s=30.0,
                 margin=0.50,
                 ls_eps=0.0,
                 theta_zero=0.785):
        super(WSINet, self).__init__()

        self.transformer = XLMRobertaModel.from_pretrained("xlm-roberta-base")
        final_in_features = self.transformer.config.hidden_size
        
        self.pooling = pooling
        self.use_fc = use_fc
    
        if use_fc:
            self.dropout = nn.Dropout(p=dropout)
            self.fc = nn.Linear(final_in_features, fc_dim)
            self.bn = nn.BatchNorm1d(fc_dim)
            self.relu = nn.ReLU()
            self._init_params()
            final_in_features = fc_dim

        self.loss_module = loss_module
        if loss_module == 'arcface':
            self.final = ArcMarginProduct(final_in_features, n_classes,
                                          s=s, m=margin, easy_margin=False, ls_eps=ls_eps)
        else:
            self.final = nn.Linear(final_in_features, n_classes)

    def _init_params(self):
        nn.init.xavier_normal_(self.fc.weight)
        nn.init.constant_(self.fc.bias, 0)
        nn.init.constant_(self.bn.weight, 1)
        nn.init.constant_(self.bn.bias, 0)

    def forward(self, input_ids,attention_mask, label, position):
        feature = self.extract_feat(input_ids,attention_mask)
        if self.loss_module == 'arcface':
            logits = self.final(feature, label)
        else:
            logits = self.final(feature)
        return logits

    def extract_feat(self, input_ids,attention_mask):
        x = self.transformer(input_ids=input_ids,attention_mask=attention_mask)
        
        features = x[0]
        features = features[:,0,:]

        if self.use_fc:
            features = self.dropout(features)
            features = self.fc(features)
            features = self.bn(features)
            features = self.relu(features)

        return features

# Training Loop

In [38]:
def train_fn(dataloader,model,criterion,device, optimizer,scheduler,epoch, mode):
    
    loss_score = AvgMeter()
    acc_meter = AvgMeter()
    #print(acc_meter)
    tk0 = tqdm(enumerate(dataloader), total=len(dataloader))
    for bi,d in tk0:
        
        batch_size = d[0].shape[0]

        input_ids = d[0]
        attention_mask = d[1]
        targets = d[2]
        position = d[3]
        #print(targets)
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        targets = targets.to(device)
        if mode == 'train':
            optimizer.zero_grad()

        output = model(input_ids,attention_mask,targets, position)
        
        loss = criterion(output,targets)
        if mode == 'train':
            loss.backward()
            optimizer.step()
        
        accuracy = get_accuracy(output.detach(), targets)
        acc_meter.update(accuracy.item(), batch_size)
        
        loss_score.update(loss.detach().item(), batch_size)
        if mode == 'train':
            tk0.set_postfix(Train_Loss=loss_score.avg,Epoch=epoch,LR=optimizer.param_groups[0]['lr'])
        else:
            tk0.set_postfix(Train_Loss=loss_score.avg,Epoch=epoch)
        if scheduler is not None:
                scheduler.step()
        
    return loss_score, acc_meter

# Engine

In [39]:
encoder = LabelEncoder()
train['labels'] = encoder.fit_transform(train['labels'])

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [40]:
train_df, valid_df = train_test_split(train, 
                                      test_size=0.33, 
                                      shuffle=True, 
                                      random_state=42, 
                                      stratify=train['labels'])

In [41]:
train['labels']

0        0
1        0
2        0
3        0
4        0
        ..
3486    46
3487    46
3488    46
3489    46
3490    46
Name: labels, Length: 2284, dtype: int64

In [42]:
len(train_df.labels.unique())

47

In [43]:
len(train_df.word.unique())

20

In [44]:
len(train_df.word.unique())

20

In [45]:
NUM_WORKERS = 4
TRAIN_BATCH_SIZE = 16
EPOCHS = 60
SEED = 2020
LR = 5e-5

device = torch.device('cuda')

################################################# MODEL ####################################################################

TOKENIZER = XLMRobertaTokenizer.from_pretrained("xlm-roberta-base")
transformer_model = XLMRobertaModel.from_pretrained("xlm-roberta-base")

################################################ Metric Loss and its params #######################################################
loss_module = 'arcface'
s = 30.0
m = 0.5 
ls_eps = 0.0
easy_margin = False

############################################################################################################################
model_params = {
    'n_classes':47,
    'model_name':transformer_model,
    'pooling':'first',
    'use_fc':False,
    'fc_dim':512,
    'dropout':0.0,
    'loss_module':loss_module,
    's':30.0,
    'margin':0.50,
    'ls_eps':0.0,
    'theta_zero':0.785
}

In [46]:
train_feats = convert_dataset_to_features(train_df)
valid_feats = convert_dataset_to_features(valid_df)

Not fitted examples percentage: 0.0
Not fitted examples percentage: 0.0


In [47]:
def fetch_loss():
    loss = nn.CrossEntropyLoss()
    return loss

In [48]:
def run():
    # Defining DataSet
    train_dataset = WSIDataset(
        train_df,
        train_feats
    )
    valid_dataset = WSIDataset(
        valid_df,
        valid_feats
    )
        
    train_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=TRAIN_BATCH_SIZE,
        pin_memory=True,
        drop_last=True,
        num_workers=NUM_WORKERS
    )
    
    valid_loader = torch.utils.data.DataLoader(
        valid_dataset,
        batch_size=TRAIN_BATCH_SIZE,
        pin_memory=True,
        drop_last=True,
        num_workers=NUM_WORKERS
    )
    
    # Defining Device
    device = torch.device("cuda")
    
    # Defining Model for specific fold
    model = WSINet(**model_params)
    model.to(device)
    
    #DEfining criterion
    criterion = fetch_loss()
    criterion.to(device)
        
    # Defining Optimizer with weight decay to params other than bias and layer norms
    param_optimizer = list(model.named_parameters())
    no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
    optimizer_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.0001},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0},
            ]  
    
    optimizer = AdamW(optimizer_parameters, lr=LR)
    
    #Defining LR SCheduler
    scheduler = get_linear_schedule_with_warmup(
        optimizer, 
        num_warmup_steps=len(train_loader)*2, 
        num_training_steps=len(train_loader)*EPOCHS
    )
        
    # THE ENGINE LOOP
    best_loss = 100000
    best_acc = 0
    for epoch in range(EPOCHS):
        model.train()
        train_loss, train_acc = train_fn(train_loader, model,criterion,  device, optimizer,scheduler=scheduler,epoch=epoch, mode='train')
        
        model.eval()
        with torch.no_grad():
            valid_loss, valid_acc = train_fn(valid_loader,model, criterion,
                                              device,optimizer=None, scheduler=None, epoch=epoch, mode='val')
        
        print(f"EPOCH {epoch} with train loss: {train_loss} and train accuracy: {train_acc}")
        print(f"EPOCH {epoch} with valid loss: {valid_loss} and valid accuracy: {valid_acc}")
        if valid_loss.avg < best_loss:
            best_loss = valid_loss.avg
            torch.save(model.state_dict(),f'xlmr_base_num_epochs_best_loss_word.bin')
        if valid_acc.avg > best_acc:
            torch.save(model.state_dict(),f'xlmr_base_num_epochs_best_acc_word.bin')
        if epoch == 8:
            torch.save(model.state_dict(),f'xlmr_base_num_epochs_epoch_8_word.bin')

In [49]:
run()

100%|██████████| 47/47 [00:03<00:00, 14.48it/s, Epoch=0, Train_Loss=12.1]


EPOCH 0 with train loss: Metric: 16.9671 and train accuracy: Metric: 0.0026
EPOCH 0 with valid loss: Metric: 12.0579 and valid accuracy: Metric: 0.0585


100%|██████████| 47/47 [00:03<00:00, 14.35it/s, Epoch=1, Train_Loss=7.32]


EPOCH 1 with train loss: Metric: 10.1926 and train accuracy: Metric: 0.2224
EPOCH 1 with valid loss: Metric: 7.3233 and valid accuracy: Metric: 0.4189


100%|██████████| 47/47 [00:03<00:00, 14.07it/s, Epoch=2, Train_Loss=5.09]


...

EPOCH 57 with train loss: Metric: 0.0279 and train accuracy: Metric: 0.9974
EPOCH 57 with valid loss: Metric: 2.1216 and valid accuracy: Metric: 0.9016


100%|██████████| 47/47 [00:03<00:00, 14.47it/s, Epoch=58, Train_Loss=2.13]


EPOCH 58 with train loss: Metric: 0.0304 and train accuracy: Metric: 0.9961
EPOCH 58 with valid loss: Metric: 2.1345 and valid accuracy: Metric: 0.9016


100%|██████████| 47/47 [00:03<00:00, 14.30it/s, Epoch=59, Train_Loss=2.12]


EPOCH 59 with train loss: Metric: 0.0481 and train accuracy: Metric: 0.9974
EPOCH 59 with valid loss: Metric: 2.1238 and valid accuracy: Metric: 0.9016


# Inference

In [50]:
# Preliminaries
from tqdm import tqdm
import math
import random
import os
import pandas as pd
import numpy as np

#torch
import torch
import torch.nn as nn
from torch.nn import Parameter
from torch.nn import functional as F
from torch.utils.data import Dataset,DataLoader

import transformers

import gc
import matplotlib.pyplot as plt
import cudf
import cuml
import cupy
from cuml import PCA
from cuml.neighbors import NearestNeighbors
from sklearn.preprocessing import Normalizer

In [498]:
NUM_WORKERS = 4
BATCH_SIZE = 16
SEED = 42

device = torch.device('cuda')

################################################  ADJUSTING FOR CV OR SUBMIT ##############################################




################################################# MODEL ####################################################################

transformer_model = 'xlm-roberta-base'
TOKENIZER = XLMRobertaTokenizer.from_pretrained(transformer_model)

################################################ MODEL PATH ###############################################################

TEXT_MODEL_PATH = './xlmr_base_num_epochs_best_loss_word.bin'

model_params = {
    'n_classes':47,
    'model_name':transformer_model,
    'pooling': 'mean',
    'use_fc':False,
    'fc_dim':512,
    'dropout':0.3,
}

In [499]:
class WSINet(nn.Module):

    def __init__(self,
                 n_classes,
                 model_name='bert-base-uncased',
                 use_fc=False,
                 fc_dim=512,
                 dropout=0.0,
                pooling='mean'):
        """
        :param n_classes:
        :param model_name: name of model from pretrainedmodels
            e.g. resnet50, resnext101_32x4d, pnasnet5large
        :param pooling: One of ('SPoC', 'MAC', 'RMAC', 'GeM', 'Rpool', 'Flatten', 'CompactBilinearPooling')
        :param loss_module: One of ('arcface', 'cosface', 'softmax')
        """
        super(WSINet, self).__init__()

        self.transformer = XLMRobertaModel.from_pretrained(model_name)
        final_in_features = self.transformer.config.hidden_size
        
        self.use_fc = use_fc
    
        if use_fc:
            self.dropout = nn.Dropout(p=dropout)
            self.fc = nn.Linear(final_in_features, fc_dim)
            self.bn = nn.BatchNorm1d(fc_dim)
            self._init_params()
            final_in_features = fc_dim


    def _init_params(self):
        nn.init.xavier_normal_(self.fc.weight)
        nn.init.constant_(self.fc.bias, 0)
        nn.init.constant_(self.bn.weight, 1)
        nn.init.constant_(self.bn.bias, 0)

    def forward(self, input_ids,attention_mask, position):
        feature = self.extract_feat(input_ids,attention_mask, position)
        return F.normalize(feature)

    def extract_feat(self, input_ids,attention_mask, position):
        x = self.transformer(input_ids=input_ids,attention_mask=attention_mask)
        #x = x[0]
        #print(x.size())
        feature = self.extract_features(x, position)
        return feature
    
    def extract_features(self, hidden_states, positions):
        pool_type = model_params['pooling']
        #merge_type = self.local_config['target_embeddings']
        #print(hidden_states)
        hidden_states = hidden_states.last_hidden_state 
        #print(type(hidden_states))
        bs, seq, hs = hidden_states.size()
        features = []
        #print(f"Positions:{positions}")
        for ex_id in range(bs):
            #print(type(positions[ex_id, 0].item()))
            #print(positions[ex_id, 0].item())
            #print(type(ex_id))
            #print(positions[ex_id])
            start1, end1 = positions[0][ex_id], positions[1][ex_id]
            if pool_type == 'mean':
                emb1 = hidden_states[ex_id, start1:end1].mean(dim=0) # hidden
                #emb2 = hidden_states[ex_id, start2:end2].mean(dim=0) # hidden
            elif pool_type == 'max':
                emb1, _ = hidden_states[ex_id, start1:end1].max(dim=0) # hidden
                #emb2, _ = hidden_states[ex_id, start2:end2].max(dim=0) # hidden
            elif pool_type == 'first':
                emb1 = hidden_states[ex_id, start1]
                #emb2 = hidden_states[ex_id, start2]
            elif pool_type == 'mf':
                embs1 = hidden_states[ex_id, start1:end1] # hidden
                #embs2 = hidden_states[ex_id, start2:end2] # hidden
                #emb1, emb2 = (torch.cat([embs.mean(dim=0), embs[0]], dim=-1) for embs in (embs1, embs2))
            elif pool_type.startswith('mm'):
                embs1 = hidden_states[ex_id, start1:end1] # hidden
                #embs2 = hidden_states[ex_id, start2:end2] # hidden
                #last = '' if len(pool_type)<3 else pool_type[2]
                #emb1, emb2 = (torch.cat([embs.mean(dim=0), embs.max(dim=0).values] + ([] if last=='' else [embs[0]] if last=='f' else [embs.min(dim=0).values]), dim=-1) for embs in (embs1, embs2))
            else:
                raise ValueError(f'wrong pool_type: {pool_type}')
            """
            if merge_type == 'featwise_mul':
                merged_feature = emb1 * emb2 # hidden
            elif merge_type == 'diff':
                merged_feature = emb1 - emb2
            elif merge_type == 'concat':
                merged_feature = torch.cat((emb1, emb2)) # 2 * hidden
            elif merge_type == 'mulnorm':
                merged_feature = (emb1 / emb1.norm(dim=-1, keepdim=True)) * (emb2 / emb2.norm(dim=-1, keepdim=True))
            elif merge_type == 'comb_cm':
                merged_feature = torch.cat((emb1, emb2, emb1 * emb2))
            elif merge_type == 'comb_cmn':
                emb1n = emb1 / emb1.norm(dim=-1, keepdim=True)
                emb2n = emb2 / emb2.norm(dim=-1, keepdim=True)
                merged_feature = torch.cat((emb1, emb2, emb1n*emb2n))
            elif merge_type == 'comb_cd':
                merged_feature = torch.cat((emb1, emb2, emb1 - emb2))
            elif merge_type == 'comb_cnmn':
                emb1n = emb1 / emb1.norm(dim=-1, keepdim=True)
                emb2n = emb2 / emb2.norm(dim=-1, keepdim=True)
                merged_feature = torch.cat((emb1n, emb2n, emb1n* emb2n))
            elif merge_type == 'comb_dmn':
                emb1n = emb1 / emb1.norm(dim=-1, keepdim=True)
                emb2n = emb2 / emb2.norm(dim=-1, keepdim=True)
                merged_feature = torch.cat((emb1 - emb2, emb1n* emb2n))
            elif merge_type == 'comb_dnmn':
                emb1n = emb1 / emb1.norm(dim=-1, keepdim=True)
                emb2n = emb2 / emb2.norm(dim=-1, keepdim=True)
                merged_feature = torch.cat((emb1n - emb2n, emb1n* emb2n))
            elif merge_type.startswith( 'dist_'):
                if 'n' in merge_type:
                    emb1n = emb1 / emb1.norm(dim=-1, keepdim=True)
                    emb2n = emb2 / emb2.norm(dim=-1, keepdim=True)
                dists = []
                if 'l1' in merge_type:        
                    dists.append(torch.norm(emb1n-emb2n if 'l1n' in merge_type else emb1-emb2, dim=-1, p=1, keepdim=True))
                if 'l2' in merge_type:        
                    dists.append(torch.norm(emb1n-emb2n if 'l2n' in merge_type else emb1-emb2, dim=-1, p=2, keepdim=True))
                if 'dot' in merge_type:        
                    dists.append((emb1n*emb2n if 'dotn' in merge_type else emb1*emb2 ).sum(dim=-1, keepdim=True))
                merged_feature = torch.cat(dists)
            """

            features.append(emb1.unsqueeze(0))
        output = torch.cat(features, dim=0) # bs x hidden
        return output

In [500]:
def get_text_embeddings(df, feats):
    embeds = []
    
    model = WSINet(**model_params)
    model.eval()
    
    model.load_state_dict(dict(list(torch.load(TEXT_MODEL_PATH).items())[:-1]))
    model = model.to(device)

    text_dataset = WSIDataset(df, feats)
    text_loader = torch.utils.data.DataLoader(
        text_dataset,
        batch_size=BATCH_SIZE,
        pin_memory=True,
        drop_last=False,
        num_workers=NUM_WORKERS
    )
    
    
    with torch.no_grad():
        for input_ids, attention_mask, _, positions in tqdm(text_loader): 
            input_ids = input_ids.cuda()
            attention_mask = attention_mask.cuda()
            feat = model(input_ids, attention_mask, positions)
            text_embeddings = feat.detach().cpu().numpy()
            embeds.append(text_embeddings)
    
    
    del model 
    text_embeddings = np.concatenate(embeds)
    print(f'Our text embeddings shape is {text_embeddings.shape}')
    del embeds
    gc.collect()
    return text_embeddings

In [501]:
lst1 = []
lst2 = []
for i, ex in X_test.iterrows():
    pos1 = ex.positions.split('-')[0]
    pos2 = ex.positions.split('-')[1]
    lst1.append(int(pos1))
    lst2.append(int(pos2))

In [502]:
X_test['start_1'] = lst1
X_test['end_1'] = lst2

In [503]:
test_feats = convert_dataset_to_features(X_test)

Not fitted examples percentage: 0.0


In [504]:
train_dataset = WSIDataset(
    train_df,
    train_feats
)
valid_dataset = WSIDataset(
    valid_df,
    valid_feats
)

test_dataset = WSIDataset(
    X_test,
    test_feats
    
)
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=TRAIN_BATCH_SIZE,
    pin_memory=True,
    drop_last=True,
    num_workers=NUM_WORKERS
)

valid_loader = torch.utils.data.DataLoader(
    valid_dataset,
    batch_size=TRAIN_BATCH_SIZE,
    pin_memory=True,
    drop_last=True,
    num_workers=NUM_WORKERS
)

test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=TRAIN_BATCH_SIZE,
    pin_memory=True,
    drop_last=True,
    num_workers=NUM_WORKERS
)

In [505]:
valid_df

,index,context_id,word,gold_sense_id,predict_sense_id,positions,context,labels,occurences,indice,start_1,end_1,new_gold,preds
0,1569,1570,мишень,1,NaN,75-81,. В своей пятой попытке Браун решил уже не исп...,20,88,13,75,81,0,0.0
1,1467,1468,мина,1,NaN,83-86,время возле храма Державной иконы Божией Матер...,18,51,12,83,86,0,0.0
2,1771,1772,оклад,1,NaN,72-77,"светских землевладельцев по 5 копеек, в 8 или ...",22,137,12,72,77,0,0.0
3,3054,3055,рысь,1,NaN,74-78,были далекие поля в зеленых и черных полосах. ...,36,82,12,74,78,0,0.0
4,2497,2498,проказа,1,NaN,73-79,лениво убирался знахаркою и выскочил с нами на...,28,95,12,73,79,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
749,3378,3379,хвост,3,NaN,78-83,"в хвост самолета, поскольку кто-то знающий гро...",43,8,12,78,83,1,17.0
750,2646,2647,пропасть,1,NaN,73-80,"все кончено, -- вместо того, чтобы бросить эту...",30,92,13,73,80,0,9.0
751,32,33,балка,1,NaN,78-82,"стене. Потолка над башней не было, он давно об...",0,81,12,78,82,0,14.0
752,3132,3133,среда,1,NaN,74-79,"подошла. А почему бы, собственно говоря, не пр...",38,13,12,74,79,0,16.0


In [506]:
valid_df = valid_df.reset_index()

In [507]:
train_df = train_df.reset_index()

In [508]:
#train_df
sense_encoder = LabelEncoder() 
for wrd in train_df.word.unique():
    idcs = train_df[train_df.word == wrd].index
    train_df.at[idcs, 'new_gold'] = sense_encoder.fit_transform(train_df.loc[idcs, 'gold_sense_id'])
    idcs = valid_df[valid_df.word == wrd].index
    valid_df.at[idcs, 'new_gold'] = sense_encoder.transform(valid_df.loc[idcs, 'gold_sense_id'])

In [509]:
train_df['new_gold'] =train_df['new_gold'].astype(int)
valid_df['new_gold'] =valid_df['new_gold'].astype(int)

In [510]:
valid_df

,level_0,index,context_id,word,gold_sense_id,predict_sense_id,positions,context,labels,occurences,indice,start_1,end_1,new_gold,preds
0,0,1569,1570,мишень,1,NaN,75-81,. В своей пятой попытке Браун решил уже не исп...,20,88,13,75,81,0,0.0
1,1,1467,1468,мина,1,NaN,83-86,время возле храма Державной иконы Божией Матер...,18,51,12,83,86,0,0.0
2,2,1771,1772,оклад,1,NaN,72-77,"светских землевладельцев по 5 копеек, в 8 или ...",22,137,12,72,77,0,0.0
3,3,3054,3055,рысь,1,NaN,74-78,были далекие поля в зеленых и черных полосах. ...,36,82,12,74,78,0,0.0
4,4,2497,2498,проказа,1,NaN,73-79,лениво убирался знахаркою и выскочил с нами на...,28,95,12,73,79,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
749,749,3378,3379,хвост,3,NaN,78-83,"в хвост самолета, поскольку кто-то знающий гро...",43,8,12,78,83,1,17.0
750,750,2646,2647,пропасть,1,NaN,73-80,"все кончено, -- вместо того, чтобы бросить эту...",30,92,13,73,80,0,9.0
751,751,32,33,балка,1,NaN,78-82,"стене. Потолка над башней не было, он давно об...",0,81,12,78,82,0,14.0
752,752,3132,3133,среда,1,NaN,74-79,"подошла. А почему бы, собственно говоря, не пр...",38,13,12,74,79,0,16.0


In [70]:
from sklearn.cluster import AgglomerativeClustering, DBSCAN, KMeans

In [368]:
from sklearn.metrics import adjusted_rand_score as ARI, silhouette_score

In [ ]:
valid_df

In [511]:
text_embeddings = get_text_embeddings(valid_df, valid_feats)

100%|██████████| 48/48 [00:03<00:00, 13.94it/s]


Our text embeddings shape is (754, 768)


In [512]:
res_lst = []
for wrd in valid_df.word.unique():
    idcs = valid_df[valid_df['word'] == wrd].index
    clustering = DBSCAN(eps=0.77,min_samples=1).fit(text_embeddings[idcs])
    valid_df.at[idcs, 'preds'] = [int(x) for x in clustering.labels_]
    res_lst.append(ARI(valid_df.new_gold[idcs], valid_df.preds[idcs]))
    print(ARI(valid_df.new_gold[idcs], valid_df.preds[idcs]))

0.8956145768993206
0.5474137931034483
0.64157824933687
1.0
0.6128500823723229
0.7097846738442052
1.0
0.9215590126985872
0.5565217391304348
0.02786618400913486
0.2817771048709517
0.0
0.2736528433076732
0.252503449984077
0.9904708563783031
-0.032948929159802305
0.45446067764998443
0.8398291510945008
0.9354120267260579
1.0


In [513]:
from statistics import mean
mean(res_lst)

0.5954172746123035

# This doesn't mean much, because it's a valid with the same classes at it was tuned on

# But here we validate on unseen classes, by GroupKFold

In [514]:
X_test = X_test.reset_index()

In [515]:
X_test

,level_0,index,context_id,word,gold_sense_id,predict_sense_id,positions,context,labels,occurences,indice,start_1,end_1,new_gold,preds
0,0,370,371,губа,1,NaN,90-93,изволили в своем любезном письме поставить мне...,12,132,13,90,93,0,0.0
1,1,371,372,губа,1,NaN,86-91,"застонала... -- Вот видишь, все приятнее и при...",12,132,13,86,91,0,0.0
2,2,372,373,губа,1,NaN,87-90,"в каждой второй девушке, красива по законам пр...",12,132,13,87,90,0,0.0
3,3,373,374,губа,1,NaN,102-104,"незабвенный Георгий! «Интересно, где он сейчас...",12,132,12,102,104,0,0.0
4,4,374,375,губа,1,NaN,81-84,". Волосы русые , глаза серые , нос прямой , на...",12,132,16,81,84,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1163,1163,3005,3006,пытка,2,NaN,107-111,президенты США от республиканцев Джон Маккейн ...,81,27,12,107,111,1,1.0
1164,1164,3006,3007,пытка,2,NaN,78-82,"кухне, доставляло радость и даже ежедневное ст...",81,27,12,78,82,1,1.0
1165,1165,3007,3008,пытка,2,NaN,57-61,", жаль, что она не умеет жить и делает для себ...",81,27,13,57,61,1,1.0
1166,1166,3008,3009,пытка,2,NaN,96-100,работать в команде. Постоянное сотрудничество ...,81,27,10,96,100,1,1.0


In [516]:
#train_df
sense_encoder = LabelEncoder() 
for wrd in X_test.word.unique():
    idcs = X_test[X_test.word == wrd].index
    X_test.at[idcs, 'new_gold'] = sense_encoder.fit_transform(X_test.loc[idcs, 'gold_sense_id'])
    #idcs = valid_df[valid_df.word == wrd].index
    #valid_df.at[idcs, 'new_gold'] = sense_encoder.transform(valid_df.loc[idcs, 'gold_sense_id'])

In [517]:
X_test['new_gold'] =X_test['new_gold'].astype(int)

In [518]:
X_test

,level_0,index,context_id,word,gold_sense_id,predict_sense_id,positions,context,labels,occurences,indice,start_1,end_1,new_gold,preds
0,0,370,371,губа,1,NaN,90-93,изволили в своем любезном письме поставить мне...,12,132,13,90,93,0,0.0
1,1,371,372,губа,1,NaN,86-91,"застонала... -- Вот видишь, все приятнее и при...",12,132,13,86,91,0,0.0
2,2,372,373,губа,1,NaN,87-90,"в каждой второй девушке, красива по законам пр...",12,132,13,87,90,0,0.0
3,3,373,374,губа,1,NaN,102-104,"незабвенный Георгий! «Интересно, где он сейчас...",12,132,12,102,104,0,0.0
4,4,374,375,губа,1,NaN,81-84,". Волосы русые , глаза серые , нос прямой , на...",12,132,16,81,84,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1163,1163,3005,3006,пытка,2,NaN,107-111,президенты США от республиканцев Джон Маккейн ...,81,27,12,107,111,1,1.0
1164,1164,3006,3007,пытка,2,NaN,78-82,"кухне, доставляло радость и даже ежедневное ст...",81,27,12,78,82,1,1.0
1165,1165,3007,3008,пытка,2,NaN,57-61,", жаль, что она не умеет жить и делает для себ...",81,27,13,57,61,1,1.0
1166,1166,3008,3009,пытка,2,NaN,96-100,работать в команде. Постоянное сотрудничество ...,81,27,10,96,100,1,1.0


In [519]:
text_embeddings = get_text_embeddings(X_test, test_feats)

100%|██████████| 73/73 [00:05<00:00, 14.27it/s]


Our text embeddings shape is (1168, 768)


In [370]:
text_embeddings[idcs]

array([[ 0.03320847, -0.03094414, -0.01723719, ...,  0.02113937,
         0.0280801 ,  0.03889019],
       [ 0.03752229, -0.05051895, -0.00842144, ...,  0.01773328,
         0.0211267 ,  0.02599461],
       [-0.04404138,  0.00218509,  0.00686565, ...,  0.07451319,
        -0.01500509,  0.04823512],
       ...,
       [-0.03242748, -0.031301  , -0.03039815, ...,  0.02284312,
        -0.02202998,  0.0473099 ],
       [-0.00055173,  0.02979285, -0.0281639 , ...,  0.02362058,
        -0.01929752,  0.02630903],
       [ 0.01139054, -0.00541821, -0.02904276, ...,  0.08239555,
        -0.03204204,  0.01985987]], dtype=float32)

In [405]:
res_lst = []
for wrd in X_test.word.unique():
    idcs = X_test[X_test['word'] == wrd].index
    clustering = AgglomerativeClustering(**study.best_params).fit(text_embeddings[idcs])
    X_test.at[idcs, 'preds'] = [int(x) for x in clustering.labels_]
    print(f"Sil{silhouette_score(text_embeddings[idcs], X_test.preds[idcs], metric='cosine')}")
    res_lst.append(ARI(X_test.new_gold[idcs], X_test.preds[idcs]))
    print(ARI(X_test.new_gold[idcs], X_test.preds[idcs]))  

Sil0.4425395131111145
0.7502517136996102
Sil0.3922174274921417
0.05123995621229751
Sil0.20700480043888092
0.03735599696150609
Sil0.41893306374549866
-0.0013646033993927682
Sil-0.07169109582901001
0.15527197768685091
Sil0.147388756275177
0.04929474352091366
Sil0.1369653344154358
-0.020529489059228148
Sil0.30020296573638916
0.5894995244091326
Sil0.3080739378929138
-0.06227119827131708
Sil0.32577013969421387
-0.024458569624315585


In [406]:
mean(res_lst)  # 0.29  

0.15242900521360572

# Get params 

In [374]:
import optuna

In [469]:

def objective(trial):
    
    params = {
        #'n_clusters':trial.suggest_int('n_clusters', 2, 15),
        'distance_threshold': trial.suggest_uniform('distance_threshold', 0.6, 0.8),  #for Euclead Agglo
        'linkage': trial.suggest_categorical('linkage', ['single', 'average', 'complete'])
    }
    sils = []
    res_lst = []
    for wrd in X_test.word.unique():
        idcs = X_test[X_test['word'] == wrd].index
        clustering = AgglomerativeClustering(n_clusters=None,**params, affinity='euclidean').fit(text_embeddings[idcs])
        X_test.at[idcs, 'preds'] = [int(x) for x in clustering.labels_]
        sils.append(silhouette_score(text_embeddings[idcs], X_test.preds[idcs], metric='euclidean'))
        res_lst.append(ARI(X_test.new_gold[idcs], X_test.preds[idcs]))
        #print(ARI(X_test.new_gold[idcs], X_test.preds[idcs]))  
    print(f"SIL={np.mean(sils)}______ARI={np.mean(res_lst)}")
    return np.mean(sils)

In [470]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

[I 2021-09-05 02:07:43,647] A new study created in memory with name: no-name-7991cbe1-d940-4f1c-855c-7d1dd1e6ad2f
[I 2021-09-05 02:07:43,901] Trial 0 finished with value: 0.12031614780426025 and parameters: {'distance_threshold': 0.6603492005565339, 'linkage': 'average'}. Best is trial 0 with value: 0.12031614780426025.


SIL=0.12031614780426025______ARI=0.047980436511744565


[I 2021-09-05 02:07:44,139] Trial 1 finished with value: 0.11256466060876846 and parameters: {'distance_threshold': 0.7922374589185095, 'linkage': 'complete'}. Best is trial 0 with value: 0.12031614780426025.


SIL=0.11256466060876846______ARI=0.029717139838401675


[I 2021-09-05 02:07:44,381] Trial 2 finished with value: 0.15502503514289856 and parameters: {'distance_threshold': 0.7567832756009578, 'linkage': 'average'}. Best is trial 2 with value: 0.15502503514289856.


...

SIL=0.21389441192150116______ARI=0.13745190147510883


[I 2021-09-05 02:08:11,031] Trial 98 finished with value: 0.21389441192150116 and parameters: {'distance_threshold': 0.7955204109052609, 'linkage': 'single'}. Best is trial 26 with value: 0.21389441192150116.


SIL=0.21389441192150116______ARI=0.13745190147510883


[I 2021-09-05 02:08:11,321] Trial 99 finished with value: 0.21377572417259216 and parameters: {'distance_threshold': 0.7882673916953812, 'linkage': 'single'}. Best is trial 26 with value: 0.21389441192150116.


SIL=0.21377572417259216______ARI=0.13793687620786269


In [459]:
print(study.best_params) # Cosine Agglo

{'n_clusters': 2, 'linkage': 'average'}


In [471]:
print(study.best_params) # euclidean Agglo

{'distance_threshold': 0.7971467801989977, 'linkage': 'single'}


In [473]:
res_lst = []
for wrd in X_test.word.unique():
    idcs = X_test[X_test['word'] == wrd].index
    clustering = AgglomerativeClustering(n_clusters=None,**study.best_params).fit(text_embeddings[idcs])
    X_test.at[idcs, 'preds'] = [int(x) for x in clustering.labels_]
    print(f"Sil{silhouette_score(text_embeddings[idcs], X_test.preds[idcs], metric='cosine')}")
    res_lst.append(ARI(X_test.new_gold[idcs], X_test.preds[idcs]))
    print(ARI(X_test.new_gold[idcs], X_test.preds[idcs]))  

Sil0.5639352202415466
0.7084971190445483
Sil0.39332571625709534
0.04639020888475886
Sil0.24402323365211487
-0.017949436369051795
Sil0.4676153063774109
-0.0013658589184143458
Sil0.1795056313276291
0.0512982204359334
Sil0.147388756275177
0.04929474352091366
Sil0.4677484631538391
0.015700442540275195
Sil0.33292356133461
0.5839815856684393
Sil0.1990835815668106
0.0019327552470202914
Sil0.3407755494117737
-0.06326076530333477


### AglomerativeClusterin best results

In [458]:
mean(res_lst)  # Cosine optuna             no lookup on ARI

0.1573302690762053

In [496]:
res_lst = []
for wrd in X_test.word.unique():
    idcs = X_test[X_test['word'] == wrd].index
    clustering = AgglomerativeClustering(**{'n_clusters': 2, 'linkage': 'average'}).fit(text_embeddings[idcs])
    X_test.at[idcs, 'preds'] = [int(x) for x in clustering.labels_]
    #print(f"Sil{silhouette_score(text_embeddings[idcs], X_test.preds[idcs], metric='cosine')}")
    res_lst.append(ARI(X_test.new_gold[idcs], X_test.preds[idcs]))
    #print(ARI(X_test.new_gold[idcs], X_test.preds[idcs]))  
print(f"Best result without looking on ARI={mean(res_lst)}")

Best result without looking on ARI=0.1573302690762053


In [497]:
from IPython.display import FileLink 
FileLink(r'./xlmr_base_num_epochs_best_acc_word.bin')

/kaggle/working/xlmr_base_num_epochs_best_acc_word.bin

In [474]:
mean(res_lst)  # euclidean optuna              No lookup on ARI

0.1374519014751088

In [ ]:
# 0.77 not Cosine
# 0.29 Cosine                   DBSCAN picked up by hand by ARI (not right)

In [ ]:
#0.154 n_clusters=None,distance_threshold=0.76, affinity='euclidean', linkage='single'    Agglo

In [ ]:
# 0.148(n_clusters=None,distance_threshold=0.3, affinity='cosine', linkage='single' Agglo

In [520]:

def objective(trial):
    
    params = {
        'n_clusters':trial.suggest_int('n_clusters', 2, 15),
        #'distance_threshold': trial.suggest_uniform('distance_threshold', 0.6, 0.8),  #for Euclead Agglo
        'linkage': trial.suggest_categorical('linkage', ['single', 'average', 'complete'])
    }
    sils = []
    res_lst = []
    for wrd in X_test.word.unique():
        idcs = X_test[X_test['word'] == wrd].index
        clustering = AgglomerativeClustering(**params, affinity='cosine').fit(text_embeddings[idcs])
        X_test.at[idcs, 'preds'] = [int(x) for x in clustering.labels_]
        sils.append(silhouette_score(text_embeddings[idcs], X_test.preds[idcs], metric='cosine'))
        res_lst.append(ARI(X_test.new_gold[idcs], X_test.preds[idcs]))
        #print(ARI(X_test.new_gold[idcs], X_test.preds[idcs]))  
    print(f"SIL={np.mean(sils)}______ARI={np.mean(res_lst)}")
    return np.mean(sils)

In [526]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=300)

[I 2021-09-05 02:26:37,094] A new study created in memory with name: no-name-8e21d754-0ae6-41d0-a899-8bf0526f92d8
[I 2021-09-05 02:26:37,265] Trial 0 finished with value: 0.14866581559181213 and parameters: {'n_clusters': 11, 'linkage': 'single'}. Best is trial 0 with value: 0.14866581559181213.
[I 2021-09-05 02:26:37,430] Trial 1 finished with value: 0.09770508110523224 and parameters: {'n_clusters': 13, 'linkage': 'single'}. Best is trial 0 with value: 0.14866581559181213.


SIL=0.14866581559181213______ARI=0.1260224942893162
SIL=0.09770508110523224______ARI=0.11361983930733217


[I 2021-09-05 02:26:37,599] Trial 2 finished with value: 0.38086259365081787 and parameters: {'n_clusters': 5, 'linkage': 'average'}. Best is trial 2 with value: 0.38086259365081787.
[I 2021-09-05 02:26:37,769] Trial 3 finished with value: 0.24749624729156494 and parameters: {'n_clusters': 7, 'linkage': 'single'}. Best is trial 2 with value: 0.38086259365081787.


SIL=0.38086259365081787______ARI=0.08594310704881734
SIL=0.24749624729156494______ARI=0.10961262842686241


[I 2021-09-05 02:26:37,940] Trial 4 finished with value: 0.2665100395679474 and parameters: {'n_clusters': 14, 'linkage': 'average'}. Best is trial 2 with value: 0.38086259365081787.
[I 2021-09-05 02:26:38,110] Trial 5 finished with value: 0.24017119407653809 and parameters: {'n_clusters': 12, 'linkage': 'complete'}. Best is trial 2 with value: 0.38086259365081787.


...

SIL=0.45778584480285645______ARI=0.083851590853293
SIL=0.4284874498844147______ARI=0.1505337999297542


[I 2021-09-05 02:27:30,689] Trial 298 finished with value: 0.36179131269454956 and parameters: {'n_clusters': 6, 'linkage': 'average'}. Best is trial 35 with value: 0.49697089195251465.


SIL=0.36179131269454956______ARI=0.10568223103683858


[I 2021-09-05 02:27:30,898] Trial 299 finished with value: 0.49697089195251465 and parameters: {'n_clusters': 2, 'linkage': 'average'}. Best is trial 35 with value: 0.49697089195251465.


SIL=0.49697089195251465______ARI=0.17311206703779042


In [527]:
study.best_params

{'n_clusters': 2, 'linkage': 'average'}

In [528]:
res_lst = []
for wrd in X_test.word.unique():
    idcs = X_test[X_test['word'] == wrd].index
    clustering = AgglomerativeClustering(**study.best_params).fit(text_embeddings[idcs])
    X_test.at[idcs, 'preds'] = [int(x) for x in clustering.labels_]
    print(f"Sil{silhouette_score(text_embeddings[idcs], X_test.preds[idcs], metric='cosine')}")
    res_lst.append(ARI(X_test.new_gold[idcs], X_test.preds[idcs]))
    print(ARI(X_test.new_gold[idcs], X_test.preds[idcs]))  

Sil0.6789548397064209
0.7387489133233567
Sil0.5370218753814697
0.016738638430073463
Sil0.5211890935897827
-0.026097388191352485
Sil0.6273921728134155
0.00018439290040459487
Sil0.34413036704063416
-0.015302404663589992
Sil0.3070247173309326
0.25051361919278947
Sil0.5830160975456238
0.0073807162465917816
Sil0.43193668127059937
0.8140763592125263
Sil0.47703659534454346
-0.047828132881502824
Sil0.4620063602924347
-0.007294043191392925


In [530]:
mean(res_lst) # Best model with no lookup on ARI {'n_clusters': 2, 'linkage': 'average'}, xlm-r smallest loss

0.17311206703779042

** Best results for my test dataset is the model with the smallest loss and AglloClustering with average linkage and n_clusters=2